In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
!which python

/opt/anaconda3/bin/python


In [3]:
from torchtext import data
from torchtext import datasets

In [4]:
import torch
from torch import nn, optim

In [6]:
import spacy
from spacy.cli.download import download
download(model="en_core_web_sm")

  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
HEADLINE = data.Field(tokenize='spacy', include_lengths = True,tokenizer_language='en_core_web_sm')
ISSARCASTIC = data.LabelField()

In [10]:
_fields = {
    'headline':('headline', HEADLINE),
    'is_sarcastic':('sarcastic', ISSARCASTIC)
}

In [11]:
sarcasm_data = data.TabularDataset(
    path='./data/Sarcasm_Headlines_Dataset.json',
    format='json',
    fields=_fields
)

In [14]:
print(vars(sarcasm_data[0]))

{'headline': ['former', 'versace', 'store', 'clerk', 'sues', 'over', 'secret', "'", 'black', 'code', "'", 'for', 'minority', 'shoppers'], 'sarcastic': 0}


In [15]:
train_data, test_data = sarcasm_data.split()

In [16]:
len(test_data)

8013

In [17]:
MAX_VOCAB_SIZE = 50_000

In [18]:
HEADLINE.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

.vector_cache/glove.6B.zip: 862MB [05:35, 2.57MB/s]                             
100%|███████████████████████████████▉| 399999/400000 [00:10<00:00, 37192.01it/s]


In [19]:
ISSARCASTIC.build_vocab(train_data)

In [20]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x:len(x.headline),
    sort_within_batch = True,
    device = device)

In [21]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional,
                          dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        return self.fc(hidden)
        

In [22]:
INPUT_DIM = len(HEADLINE.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = HEADLINE.vocab.stoi[HEADLINE.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,494,157 trainable parameters


In [24]:
pretrained_embeddings = HEADLINE.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([21835, 100])


In [25]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.3767e-01,  2.0080e-01, -1.0861e+00,  ..., -2.5730e+00,
         -5.5604e-02, -1.6429e+00],
        [-1.2565e+00,  9.3560e-01,  1.1847e-01,  ..., -1.2946e+00,
          2.2023e-01, -1.4333e-01],
        [-1.8970e-01,  5.0024e-02,  1.9084e-01,  ..., -3.9804e-01,
          4.7647e-01, -1.5983e-01],
        ...,
        [ 5.8076e-01, -1.5075e-01, -2.1487e+00,  ..., -4.1770e-01,
         -1.8682e-01,  6.1738e-01],
        [ 1.1097e-01,  3.6155e-01,  2.0370e-01,  ...,  2.2696e-01,
          3.0238e-01, -5.6025e-04],
        [ 1.6229e-01, -1.0102e+00,  3.4051e-02,  ...,  4.5540e-01,
          1.1883e+00, -1.9166e+00]])

In [26]:
UNK_IDX = HEADLINE.vocab.stoi[HEADLINE.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.8970e-01,  5.0024e-02,  1.9084e-01,  ..., -3.9804e-01,
          4.7647e-01, -1.5983e-01],
        ...,
        [ 5.8076e-01, -1.5075e-01, -2.1487e+00,  ..., -4.1770e-01,
         -1.8682e-01,  6.1738e-01],
        [ 1.1097e-01,  3.6155e-01,  2.0370e-01,  ...,  2.2696e-01,
          3.0238e-01, -5.6025e-04],
        [ 1.6229e-01, -1.0102e+00,  3.4051e-02,  ...,  4.5540e-01,
          1.1883e+00, -1.9166e+00]])


In [27]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [28]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [29]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    return correct.sum() / len(correct)

In [30]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0.0
    epoch_acc = 0.0
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.headline
        
        predictions = model(text, text_lengths).squeeze(1)
#         print(predictions)
        loss = criterion(predictions, batch.sarcastic.type_as(predictions))
        acc = binary_accuracy(predictions, batch.sarcastic.type_as(predictions))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [32]:
N_EPOCHS = 5


for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 52s
	Train Loss: 0.502 | Train Acc: 74.98%
Epoch: 02 | Epoch Time: 2m 2s
	Train Loss: 0.351 | Train Acc: 84.66%
Epoch: 03 | Epoch Time: 1m 57s
	Train Loss: 0.289 | Train Acc: 87.62%
Epoch: 04 | Epoch Time: 1m 48s
	Train Loss: 0.241 | Train Acc: 90.12%
Epoch: 05 | Epoch Time: 1m 28s
	Train Loss: 0.208 | Train Acc: 91.69%


In [33]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.headline
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.sarcastic.type_as(predictions))
            
            acc = binary_accuracy(predictions, batch.sarcastic.type_as(predictions))

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
# model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.259 | Test Acc: 89.35%


In [36]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [37]:
def predict_sarcasm(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [HEADLINE.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [38]:
predict_sarcasm(model, "She gave him a sarcastic smile.")

0.31399819254875183